<a href="https://colab.research.google.com/github/saurabh-singh-rajput/commit-message-generator/blob/main/commit_message_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Automated commit message generation

In [ ]:
# Clone the repo incase running on colab, else comment it if directly being run on local GPU.
!git clone https://github.com/saurabh-singh-rajput/commit-message-generator.git

Cloning into 'commit-message-generator'...
remote: Enumerating objects: 409, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 409 (delta 15), reused 0 (delta 0), pack-reused 377
Receiving objects: 100% (409/409), 61.34 MiB | 9.03 MiB/s, done.
Resolving deltas: 100% (130/130), done.


In [ ]:
# move to the cloned repo directory
%cd commit-message-generator

/content/commit-message-generator


In [ ]:
# Install required dependencies
!pip install accelerate -U
!pip install dataset evaluate
!pip install rouge_score
!pip install transformers[onnx]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.23
    Uninstalling SQLAlchemy-2.0.23:
      Successfully uninstalled SQLAlchemy-2.0.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ip

### Read filtered data

This **filtered_data.csv** is the result of passing around ~63000 data points through the filter proposed in this [paper](https://arxiv.org/pdf/2202.02974.pdf). The filter retains the good quality messages that contain the aspect of both *why* and *what* from the code changes made into the message.

In [ ]:
import pandas as pd
data = pd.read_csv("filtered_data.csv")

### Preprocess data

We read the diff and messages tables seperately and then combine them into one dataframe : "data"

In [ ]:
data.head()

,Unnamed: 0,diff,message
0,0,mmm a / README . md <nl> ppp b / README . md <...,updated read me more documentation to come !
1,1,Binary files a / bigbluebutton - client / bran...,Updated fit - to - screen icon
2,2,mmm a / util - taglib / src / com / liferay / ...,LPS - 64187 update package info
3,3,mmm a / bindings / cpp / configure . ac <nl> p...,Remove unneeded check for cppunit
4,4,mmm a / owncloud - android - library <nl> ppp ...,Updated library to fix bug in SAML authenticat...


In [ ]:
# Dropping the unnecessary index column: "Unnamed: 0"

data.drop(data.columns[0], inplace=True, axis=1)

In [ ]:
data.head()

,diff,message
0,mmm a / README . md <nl> ppp b / README . md <...,updated read me more documentation to come !
1,Binary files a / bigbluebutton - client / bran...,Updated fit - to - screen icon
2,mmm a / util - taglib / src / com / liferay / ...,LPS - 64187 update package info
3,mmm a / bindings / cpp / configure . ac <nl> p...,Remove unneeded check for cppunit
4,mmm a / owncloud - android - library <nl> ppp ...,Updated library to fix bug in SAML authenticat...


In [ ]:
# Renaming the column

data.rename(columns = {'diff':'commits'}, inplace=True)

In [ ]:
# Checking for any missing values

data.isna().any()

commits    False
message    False
dtype: bool

### Train and test splits

85% of the data is for training

15% of the data is for validation

We have a seperate set for testing the data

In [ ]:
val_data = data.iloc[:int(len(data)*0.085),:]
train_data = data.iloc[int(len(data)*0.085):,:]

Resetting indexes after the split

In [ ]:
val_data.reset_index(drop=True, inplace=True)
train_data.reset_index(drop=True, inplace=True)

In [ ]:
type(val_data)

pandas.core.frame.DataFrame

In [ ]:
print("Train and validation data length",len(train_data)," ",len(val_data))

Train and validation data length 27707   2573


Now, we have train and test dataset: each dataset consists of code-diff data and commit messages.

### Model

Encoder-Decoder Model from Hugginface. The inspiration for this comes from this [paper](https://arxiv.org/abs/1907.12461)

We will be using graphcodeBERT as an Encoder and GPT-2 as Decoder. We then finetune this on our dataset of commits and corresponding messages.

Importing libraries : AutoTokenizer for tokenizing the input according to codebert input format and EncoderDecoder model to construct the entire model

In [ ]:
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel

### Initialize the model with encoder and decoder

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("microsoft/graphcodebert-base", "gpt2")

config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.9.ln_cross_attn.bias', 'h.6.ln_cross_attn.weight', 'h.6.crossattention.c_attn.bias', 'h.4.crossattention.q_attn.weight', 'h.10.crossattention.q_attn.weight', 'h.6.crossattention.c_proj.bias', 'h.2.crossattention.c_proj.weight', 'h.2.ln_cross_attn.weight', 'h.9.crossattention.c_proj.bias', 'h.5.crossattention.c_attn.bias', 'h.9.crossattention.q_attn.bias', 'h.10.ln_cross_attn.bias', 'h.10.crossattention.c_attn.weight', 'h.5.crossattention.c_proj.bias', 'h.1.ln_cross_attn.weight', 'h.9.crossattention.q_attn.weight', 'h.10.crossattention.c_attn.bias', 'h.8.crossattention.c_proj.bias', 'h.8.crossattention.c_attn.weight', 'h.0.crossattention.c_attn.bias', 'h.3.crossattention.q_attn.weight', 'h.2.crossattention.c_attn.bias', 'h.7.crossattention.c_attn.bias', 'h.3.ln_cross_attn.bias', 'h.11.crossattention.c_attn.bias', 'h.4.crossattention.c_attn.weight', 'h.8.crossattention.c_

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### Set the model in training mode

In [ ]:
# We will do .train() to set the model in training model. If the model is not in training mode, the weights will not be updates. So, its not learning anything

model.train()

# Please note that model.eval() will be used while inference and evaluation stage

EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

In [ ]:
# Total number of parameters of the model

model.num_parameters()

277452288

### Load codebert's tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1. Setting cls_token : To indicate start of sentence
2. Setting sep_token: To indicate end of sentence

#### The imports are for the following reasons:

1. The datasets is used to transform pandas dataframe to pyarrow (convinient to use with transformers)
2. The rouge_score and evalute libraries are for evalutaion metrics

In [ ]:
# Importing the libraries

import datasets
import evaluate

In [ ]:
# Loading the metric from huggingface datasets library to evaluate the model performance

rouge = datasets.load_metric("rouge")
bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")

<ipython-input-21-1ebb00601f10>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


We define a function to evaluate rouge score and bleu-variants. This is optinal but gives us a better evaluation criteria for the model

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
    bleu1 = bleu.compute(predictions=pred_str, references=label_str,max_order=1)
    bleu2 = bleu.compute(predictions=pred_str, references=label_str, max_order=2)
    bleu3 = bleu.compute(predictions=pred_str, references=label_str, max_order=3)
    bleu4 = bleu.compute(predictions=pred_str, references=label_str, max_order=4)


    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
        "bleu1_score":round(bleu1["bleu"],4),
        "bleu2_score":round(bleu2["bleu"],4),
        "bleu3_score":round(bleu3["bleu"],4),
        "bleu4_score":round(bleu4["bleu"],4)
        }

We define a function to process the input data : code-diff files.

The following are the steps involved:

1. It takes an input batch (16 (correlate with RAM)) from dataset.
2. The code-diff files and messages are tokenized using the codebert tokenizer.
3. Labels is just a copy of commit messages tokenized id's.
4. pad_token ids are set to -100 so that they are not considered during training and evaluation (high-level intuition).

Things to note : A maximum size of 123 is considered for code-diff files and a maximum of 30 for commit messages (The values are based on the maximum tokens present in the set)

**Hyper-parameters :**

1. batch_size
2. encoder_max_length
3. decoder_max_length

In [ ]:
batch_size=16
encoder_max_length=123
decoder_max_length=50

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["commits"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["message"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask

  #This was required in the earlier version, but internal implementation captures this so we don't have to provide
  # batch["decoder_input_ids"] = outputs.input_ids
  # batch["decoder_attention_mask"] = outputs.attention_mask

  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

### Set the model configuration

This entirely depends on the type of encoder and decoder

In [ ]:
# set special tokens
# We are setting the entire models special tokens
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
model.config.vocab_size = model.config.decoder.vocab_size # Setting the models' vocabulary to GPT-2's vocabulary size (~50K)
model.config.max_length = 64 # Maximum length of message generated
model.config.min_length = 5 # Minimum length of message generated
model.config.no_repeat_ngram_size = 3 # A word will not be repeated more than three times while geenrating a new commit message
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4 # Top-4 words for beam search rather than considering all words for greedy search

### Dataset restructuring

We are doing this to convert out pandas dataframe to pyarrow dataset. Pyarrow has good functionality and alignment with hugginface datasets library and we could use map function to manipulate the whole dataset according a fucntion provided.

Overall, we could just write a manual function for preprocessing and use map function to extract all the input_ids and attention_masks.

In [ ]:
import pyarrow as pa
from datasets import Dataset

In [ ]:
train_set = Dataset(pa.Table.from_pandas(train_data))

In [ ]:
val_set = Dataset(pa.Table.from_pandas(val_data))

In [ ]:
type(val_set)

datasets.arrow_dataset.Dataset

### Data mapping and batch conversions according to the function
This section take the data and maps the data according to the funtion (1st argument) behaviour. We also define the batch size which is a hyperparameter

In [ ]:
train_dataset = train_set.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["commits", "message"]
)


train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)

Map:   0%|          | 0/27707 [00:00<?, ? examples/s]

In [ ]:
val_dataset = val_set.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["commits", "message"]
)


val_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)

Map:   0%|          | 0/2573 [00:00<?, ? examples/s]

Now, we have all the input_ids, attention_masks and labels for train and validation set

### We are using a trainer from Hugginface
We are using **Seq2SeqTrainingArguments** and **Seq2SeqTrainer** from Hugginface.

**trainin_args** variable holds the hyperparameters. Feel free to explore all the hyperparameters on huggingface website

trainer variable is to initialize the trianer and pass model, datasets and metric funtion that we defined.

In [ ]:
# !pip install transformers[torch]

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    overwrite_output_dir=True, # Overwrites previously saved model
    learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_steps=1_000,
    # Batch size for train and validaiton.
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir = "./", # Saves the model to current directory
    save_steps=100, # Save the model to output directory after every 500 steps
    eval_steps=300,
    warmup_steps=200,
    save_total_limit=1, # Saves one file.
    num_train_epochs=0.2, # 0.2 epochs. Increase
)


trainer = Seq2SeqTrainer(
    model=model, # Modelname
    tokenizer=tokenizer, # Tokenzier
    args=training_args, # Hyperparameter arguments
    compute_metrics=compute_metrics, # Not mandatory (Manually defined rouge metric function)
    # Datasets
    train_dataset=train_dataset,
    eval_dataset=val_dataset
    )

In [ ]:
# call .train on trainer to start training.

trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:619: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:639: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model traine

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure,Bleu1 Score,Bleu2 Score,Bleu3 Score,Bleu4 Score
300,No log,4.703175,0.103000,0.100900,0.099900,0.194500,0.145800,0.119400,0.106100


Removed shared tensor {'decoder.lm_head.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:619: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:639: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created

TrainOutput(global_step=347, training_loss=6.118840198351945, metrics={'train_runtime': 577.9245, 'train_samples_per_second': 9.588, 'train_steps_per_second': 0.6, 'total_flos': 815662345789440.0, 'train_loss': 6.118840198351945, 'epoch': 0.2})

## Testing the trained model

It follows similar steps as previous except that we are not training the model, but just using the trained model to produce results and compare

### Preprocessing

In [ ]:
diff = pd.read_csv("test.3000.diff", sep="/n")

<ipython-input-33-de9b67a6c597>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  diff = pd.read_csv("test.3000.diff", sep="/n")


In [ ]:
mess = pd.read_csv("test.3000.msg", sep="/n")

<ipython-input-34-0b6aac626690>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  mess = pd.read_csv("test.3000.msg", sep="/n")


In [ ]:
diff.rename(columns={diff.columns[0]: "commits"}, inplace=True)
mess.rename(columns={mess.columns[0]: "message"}, inplace=True)

In [ ]:
# Taking 10% fraction of samples to generate, on colab free tier(less time)
diff_sample = diff.sample(frac=0.1, random_state=42).reset_index(drop=True)
mess_sample = mess.sample(frac=0.1, random_state=42).reset_index(drop=True)

In [ ]:
diff_sample["commits"][6]

'mmm a / plugins / groovy / test / org / jetbrains / plugins / groovy / lang / resolve / ResolvePropertyTest . groovy <nl> ppp b / plugins / groovy / test / org / jetbrains / plugins / groovy / lang / resolve / ResolvePropertyTest . groovy <nl> print ba < caret > r <nl> " " " ) <nl> def ref = findReference ( ) <nl> def resolved = ref . resolve ( ) <nl> - print resolved . class <nl> assertInstanceOf resolved , GrAccessorMethod . class <nl> } <nl>'

### converting to pyarrow dataset

In [ ]:
val = Dataset(pa.Table.from_pandas(diff_sample))

In [ ]:
import datasets
from transformers import AutoTokenizer, EncoderDecoderModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("checkpoint-300") # Please find the checkpoint save in the current directotry
model = EncoderDecoderModel.from_pretrained("checkpoint-300")

# Loading the model to gpu
model.to()

EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

In [ ]:
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["commits"], padding="max_length", truncation=True, max_length=123, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

Increase the batch size if you want faster computation of ids and attetnion_mask.

In [ ]:
results = val.map(generate_summary, batched=True, batch_size=32)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
# Create a DataFrame from the predicted and original messages
info_df = pd.DataFrame({
    'Predicted Message': results["pred"],
    'Original Message': mess_sample["message"]
})

# Use the .head() method to print the top rows of the DataFrame
info_df.head(300)

,Predicted Message,Original Message
0,LPS -.,Turn off PCA test - > breaks all successive JU...
1,Update README. md,Updated changelog
2,Update version to 1.,Fix Travis again and again .
3,Update README.,Remove plugins / from gitignore
4,LPS -.,Added missing @ Override annotation in NearCac...
...,...,...
295,Update version to 1. 0. 0,adjust the Maven version number
296,LPS -.,Fixed path issue
297,prepare version to 1. 0. 0,Set version to 3 . 3 . 0 from 3 . 3 . 0a
298,LPS -.,Fixing transitive - ness of provided


In [ ]:
pred_str = results["pred"]
label_str = mess_sample["message"]

rouge_output = rouge.compute(predictions=pred_str, references=label_str)
bleu1 = bleu.compute(predictions=pred_str, references=label_str,max_order=1)
bleu2 = bleu.compute(predictions=pred_str, references=label_str, max_order=2)
bleu3 = bleu.compute(predictions=pred_str, references=label_str, max_order=3)
bleu4 = bleu.compute(predictions=pred_str, references=label_str, max_order=4)
meteor = meteor.compute(predictions=pred_str, references=label_str)

print("Rouge: ",rouge_output)
print("Bleu-1: ", bleu1["bleu"])
print("Bleu-2: ",bleu2["bleu"])
print("Bleu-3: ",bleu3["bleu"])
print("Bleu-4: ", bleu4["bleu"])
print("Meteor: ",meteor["meteor"])

Rouge:  {'rouge1': AggregateScore(low=Score(precision=0.14935119047619042, recall=0.11414809981684984, fmeasure=0.1210736782710466), mid=Score(precision=0.17936111111111108, recall=0.14120019332519332, fmeasure=0.14874984591431958), high=Score(precision=0.21397023809523805, recall=0.17198295940170946, fmeasure=0.17976238920679696)), 'rouge2': AggregateScore(low=Score(precision=0.0743277777777778, recall=0.06704631433381432, fmeasure=0.07005545935545938), mid=Score(precision=0.10105555555555558, recall=0.09174194324194324, fmeasure=0.09528542753542757), high=Score(precision=0.13033888888888892, recall=0.11975454545454547, fmeasure=0.12333673641173643)), 'rougeL': AggregateScore(low=Score(precision=0.14291964285714293, recall=0.11130313644688641, fmeasure=0.11698687314147843), mid=Score(precision=0.17696825396825394, recall=0.13904634717134717, fmeasure=0.14631999141209664), high=Score(precision=0.21239960317460305, recall=0.17086826159951166, fmeasure=0.17935509256825047)), 'rougeLsum':